Unir Excels

In [10]:

import pandas as pd
import glob
import re

In [17]:
def clean_column_names(df):
    """Limpia los nombres de columnas eliminando saltos de línea y caracteres especiales"""
    new_columns = []
    for col in df.columns:
        # Convertir a string y eliminar saltos de línea
        clean_col = str(col).replace('\n', ' ').replace('\r', ' ')
        # Eliminar espacios múltiples
        clean_col = re.sub(r'\s+', ' ', clean_col)
        # Eliminar números entre paréntesis al final: (11), (1), etc.
        clean_col = re.sub(r'\s*\(\d+\)\s*$', '', clean_col)
        # Eliminar comillas y caracteres especiales al inicio/final
        clean_col = clean_col.strip().strip('"').strip("'")
        # Limpiar espacios finales otra vez después de eliminar paréntesis
        clean_col = clean_col.strip()
        clean_col = clean_col.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
        clean_col = clean_col.replace('Á', 'a').replace('É', 'e').replace('Í', 'i').replace('Ó', 'o').replace('Ú', 'u')
        clean_col = clean_col.replace('ñ', 'n').replace('Ñ', 'n')
        
        # Convertir a minúsculas
        clean_col = clean_col.lower()
        
        #new_columns.append(clean_col)
        new_columns.append(clean_col)
    
    print(new_columns)
    df.columns = new_columns
    return df

In [19]:
def separate_excel_sheets(file_path):
    """Une todas las hojas de un Excel y limpia los nombres de columnas"""
    try:
        # Leer el archivo Excel
        xls = pd.ExcelFile(file_path)
        print(f"Hojas encontradas: {xls.sheet_names}")
        
        all_data = []
        
        for sheet_name in xls.sheet_names:
            print(f"Procesando hoja: {sheet_name}")
            
            # Leer la hoja
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            
            # Limpiar nombres de columnas
            df = clean_column_names(df)
            
            # Agregar columna de origen
            df['sheet_origen'] = sheet_name
            
            all_data.append(df)
            print(f"  - Filas: {len(df)}, Columnas: {len(df.columns)}")
        
        return all_data
    except Exception as e:
        print(f"Error procesando el archivo: {e}")
        return None

In [32]:
# En base a los dataset generados -> name of sheet sera mi nueva columna donde juntare la suma de todos los tipos de carro
# dataset nuevo: ubicacion interseccion via toma, sentido, dia , hora , cantidad de carros

def simple_dataset_aforo(result_df_list, list_vehicles):
    """Genera dataset simplificado con columnas básicas y suma de vehículos específicos"""
    
    all_data = []
    
    for df in result_df_list:
        # Crear dataset básico
        simple_data = {}
        
        # Extraer columnas básicas
        simple_data['ubicacion'] = df['ubicacion']
        simple_data['interseccion'] = df['interseccion (referencia)']
        simple_data['via de toma de informacion'] = df['via de toma de informacion']
        simple_data['sentido'] = df['sentido']
        simple_data['dia'] = df['dia']
        simple_data['hora'] = df['hora']
        simple_data['fecha'] = df['fecha']
        # Sumar vehículos específicos
        vehicle_cols = [col for col in df.columns 
                       if any(vehicle in col for vehicle in list_vehicles)]
        
        if vehicle_cols:
            vehicle_data = df[vehicle_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
            simple_data['cantidad_carros'] = vehicle_data.sum(axis=1)
        else:
            simple_data['cantidad_carros'] = 0
            
        # Origen del sheet
        simple_data['sheet_origen'] = df['sheet_origen'].iloc[0] if 'sheet_origen' in df.columns else 'Unknown'
        
        # Crear DataFrame
        simple_df = pd.DataFrame(simple_data)
        all_data.append(simple_df)
   
    
    return all_data

In [26]:
# Retorna un dataframe combinado
def combined_dataset(datasets):
    final_df = pd.concat(datasets, ignore_index=True)
    return final_df
    

In [29]:
def aforo_concat_excels(path_files: str, name_file: str, list_vehicles:str):
    """Une todos los archivos Excel de una carpeta, limpia columnas y combina sheets"""
    files = glob.glob(path_files)
    print(f"Archivos encontrados: {len(files)}")
    print("Archivos:", files)
    
    all_combined_data = []

    for file in files:
        print(f"\nProcesando archivo: {file}")
        
        # Usar la función separate_excel_sheets para cada archivo
        result_df_list = separate_excel_sheets(file)
        results_simple_dfs = simple_dataset_aforo(result_df_list, list_vehicles)
        final_df = combined_dataset(results_simple_dfs)
            
        all_combined_data.append(final_df)
        # print(f"  - Total filas del archivo: {len(file_combined)}")

    # Combinar todos los archivos
    if all_combined_data:
        final_data = pd.concat(all_combined_data, ignore_index=True)
        
        # Guardar como CSV
        final_data.to_csv(name_file, index=False)
        
        print(f"\nDataset final guardado: {name_file}")
        print(f" Total archivos procesados: {len(files)}")
        print(f" Total filas: {len(final_data)}")
        print(f" Total columnas: {len(final_data.columns)}")
        
        return final_data
    else:
        print("No se pudieron procesar los archivos")
        return None

In [25]:
file_path = "/home/jazmin/Escritorio/AlertaVIAl/BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_1.xlsx"  
result_df = separate_excel_sheets(file_path)
list_veh = ['auto privado', 'camineta pick up', 'taxi', 'mototaxi', 'moto lineal', 'bicicletas', 'scooter', 'transporte escolar o de personal', 'omnibus bus inter- provincial', 'auto colectivo', 'camioneta rural', 'micro minibus', 'bus', 'articulado corredor rojo', 'c2 ligeros', 'c2 pesados (2 - 4 ejes)', 'c3 o + pesado (semitrailer - trailer)', 'corredor rojo bus', 'corredor azul bus', 'corredor azul microbus']
#print(result_df)
results_df = simple_dataset_aforo(result_df, list_vehicles=list_veh)
#Ver el resultado
for i in results_df:
    print("Primeras 5 filas")
    print(i.head())



Hojas encontradas: ['E-O', 'O-E']
Procesando hoja: E-O
['ubicacion', 'interseccion (referencia)', 'via de toma de informacion', 'sentido', 'fecha', 'dia', 'hora', 'auto privado', 'camineta pick up', 'taxi', 'mototaxi', 'moto lineal', 'bicicletas', 'scooter', 'transporte escolar o de personal', 'omnibus bus inter- provincial', 'auto colectivo', 'camioneta rural', 'micro minibus', 'bus', 'articulado corredor rojo', 'c2 ligeros', 'c2 pesados (2 - 4 ejes)', 'c3 o + pesado (semitrailer - trailer)', 'corredor rojo bus', 'corredor azul bus', 'corredor azul microbus']
  - Filas: 8, Columnas: 28
Procesando hoja: O-E
['ubicacion', 'interseccion (referencia)', 'via de toma de informacion', 'sentido', 'fecha', 'dia', 'hora', 'auto privado', 'camineta pick up', 'taxi', 'mototaxi', 'moto lineal', 'bicicletas', 'scooter', 'transporte escolar o de personal', 'omnibus bus inter- provincial', 'auto colectivo', 'camioneta rural', 'micro minibus', 'bus', 'articulado corredor rojo', 'c2 ligeros', 'c2 pesad

In [27]:
df_com = combined_dataset(results_df)
print(df_com)

                     ubicacion                            interseccion  \
0   PUNTO 1 - VICTOR MALASQUEZ  AV. VICTOR MALASQUEZ & AV. CIENEGUILLA   
1   PUNTO 1 - VICTOR MALASQUEZ  AV. VICTOR MALASQUEZ & AV. CIENEGUILLA   
2   PUNTO 1 - VICTOR MALASQUEZ  AV. VICTOR MALASQUEZ & AV. CIENEGUILLA   
3   PUNTO 1 - VICTOR MALASQUEZ  AV. VICTOR MALASQUEZ & AV. CIENEGUILLA   
4   PUNTO 1 - VICTOR MALASQUEZ  AV. VICTOR MALASQUEZ & AV. CIENEGUILLA   
5   PUNTO 1 - VICTOR MALASQUEZ  AV. VICTOR MALASQUEZ & AV. CIENEGUILLA   
6   PUNTO 1 - VICTOR MALASQUEZ  AV. VICTOR MALASQUEZ & AV. CIENEGUILLA   
7   PUNTO 1 - VICTOR MALASQUEZ  AV. VICTOR MALASQUEZ & AV. CIENEGUILLA   
8   PUNTO 1 - VICTOR MALASQUEZ  AV. VICTOR MALASQUEZ & AV. CIENEGUILLA   
9   PUNTO 1 - VICTOR MALASQUEZ  AV. VICTOR MALASQUEZ & AV. CIENEGUILLA   
10  PUNTO 1 - VICTOR MALASQUEZ  AV. VICTOR MALASQUEZ & AV. CIENEGUILLA   
11  PUNTO 1 - VICTOR MALASQUEZ  AV. VICTOR MALASQUEZ & AV. CIENEGUILLA   
12  PUNTO 1 - VICTOR MALASQUEZ  AV. VI

In [33]:
aforo_concat_excels("../../BD_InnovaThon/Aforo_por_tramos/*.xlsx", "dataset_aforo_por_tramos.xlsx", list_vehicles=list_veh)

Archivos encontrados: 20
Archivos: ['../../BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_12.xlsx', '../../BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_5.xlsx', '../../BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_10.xlsx', '../../BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_18.xlsx', '../../BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_6.xlsx', '../../BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_7.xlsx', '../../BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_9.xlsx', '../../BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_14.xlsx', '../../BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_15.xlsx', '../../BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_16.xlsx', '../../BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_17.xlsx', '../../BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_3.xlsx', '../../BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_20.xlsx', '../../BD_InnovaThon/Aforo_por_tramos/Aforos por tramos_19.xlsx', '../../BD_InnovaThon/Aforo_por_tramos/Aforos 

,ubicacion,interseccion,via de toma de informacion,sentido,dia,hora,fecha,cantidad_carros,sheet_origen
0,PUNTO 3 - AMANCAES,AV. AMANCES & AV. SAMUEL ALCÁZAR,AV. SAMUEL ALCÁZAR,N-S,Viernes,07:00 - 07:15,2024-10-25,270.0,N-S
1,PUNTO 3 - AMANCAES,AV. AMANCES & AV. SAMUEL ALCÁZAR,AV. SAMUEL ALCÁZAR,N-S,Viernes,07:15 - 07:30,2024-10-25,336.0,N-S
2,PUNTO 3 - AMANCAES,AV. AMANCES & AV. SAMUEL ALCÁZAR,AV. SAMUEL ALCÁZAR,N-S,Viernes,07:30 - 07:45,2024-10-25,308.0,N-S
3,PUNTO 3 - AMANCAES,AV. AMANCES & AV. SAMUEL ALCÁZAR,AV. SAMUEL ALCÁZAR,N-S,Viernes,07:45 - 08:00,2024-10-25,329.0,N-S
4,PUNTO 3 - AMANCAES,AV. AMANCES & AV. SAMUEL ALCÁZAR,AV. SAMUEL ALCÁZAR,N-S,Viernes,08:00 - 08:15,2024-10-25,335.0,N-S
...,...,...,...,...,...,...,...,...,...
352,PUNTO 1 - VICTOR MALASQUEZ,AV. VICTOR MALASQUEZ & AV. CIENEGUILLA,AV. CIENEGUILLA,O-E,Martes,07:45 - 08:00,2024-11-05,255.0,O-E
353,PUNTO 1 - VICTOR MALASQUEZ,AV. VICTOR MALASQUEZ & AV. CIENEGUILLA,AV. CIENEGUILLA,O-E,Martes,08:00 - 08:15,2024-11-05,250.0,O-E
354,PUNTO 1 - VICTOR MALASQUEZ,AV. VICTOR MALASQUEZ & AV. CIENEGUILLA,AV. CIENEGUILLA,O-E,Martes,08:15 - 08:30,2024-11-05,276.0,O-E
355,PUNTO 1 - VICTOR MALASQUEZ,AV. VICTOR MALASQUEZ & AV. CIENEGUILLA,AV. CIENEGUILLA,O-E,Martes,08:30 - 08:45,2024-11-05,258.0,O-E


In [ ]:
concat_excels("../../BD_InnovaThon/Informe_de_trayectos_GPS/*.xlsx", "dataset_encuestas.xlsx")